In [1]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

2025-01-22 21:33:22.679363: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-22 21:33:22.769173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737561802.808723    6029 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737561802.820075    6029 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-22 21:33:22.915395: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import tensorflow

(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.mnist.load_data()
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.uint8)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.uint8)
x_train = x_train
x_test = x_test

In [3]:
class Model(nn.Module):
    def __init__(self, patience=4, dropout=0):
        super(Model, self).__init__()
        self.ff1 = nn.Linear(784, 100)
        self.ff2 = nn.Linear(100, 100)
        self.ff3 = nn.Linear(100, 100)
        self.output = nn.Linear(100, 10)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.loss = nn.CrossEntropyLoss()
        self.trainloader = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(x_train, y_train),
            batch_size=1000,
            shuffle=True,
        )
        self.validloader = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(x_test, y_test),
            batch_size=1000,
            shuffle=True,
        )
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        self.lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode="min", factor=0.5, patience=15
        )
        self.early_stopping_patience = patience
        self.early_stopping_counter = 0
        self.early_stopping_best_loss = float("inf")

    def forward(self, x):
        x = self.ff1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.ff2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.ff3(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

    def early_stopping(self, val_loss):
        if val_loss < self.early_stopping_best_loss:
            self.early_stopping_best_loss = val_loss
            self.early_stopping_counter = 0
        else:
            self.early_stopping_counter += 1
            if self.early_stopping_counter >= self.early_stopping_patience:
                return True
        return False



    def train_loop(self, run_name, epochs=100, l1=False, l1_lambda=0.0001, early_stopping=False):
        previous_lr = self.optimizer.param_groups[0]["lr"]
        writer = SummaryWriter(log_dir=f"runs/{run_name}")
        for i in range(epochs):
            t = 0
            self.train()
            train_progress = tqdm(
                enumerate(self.trainloader),
                desc=f"Epoch {i} - Training",
                total=len(self.trainloader),
            )
            for j, k in train_progress:
                image, label = k
                image, label = image.to("cuda"), label.to("cuda")
                self.optimizer.zero_grad()
                output = self(image.view(-1, 784))
                l = self.loss(output, label)
                if l1:
                    l1_norm = sum(p.abs().sum() for p in self.parameters())
                    l += l + l1_lambda * l1_norm
                l.backward()
                self.optimizer.step()
                t += l.item()
                train_progress.set_postfix(loss=t / (j + 1))
            t /= len(self.trainloader)
            val_loss = self.val_loop(i)
            if early_stopping and self.early_stopping(val_loss):
                print("No improvement, early stopping")
                break
            self.lr_scheduler.step(val_loss)
            writer.add_scalars("Losses", {"Train": t, "Validation": val_loss}, i)
            writer.flush()
            current_lr = self.optimizer.param_groups[0]["lr"]
            if current_lr != previous_lr:
                print(f"Learning Rate changed to {current_lr}")
                previous_lr = current_lr
        writer.close()

    def val_loop(self, i):
        t = 0
        self.eval()
        val_progress = tqdm(
            enumerate(self.validloader),
            desc=f"Epoch {i} - Validation",
            total=len(self.validloader),
        )
        correct = 0
        for j, k in val_progress:
            image, label = k
            image, label = image.to("cuda"), label.to("cuda")
            output = self(image.view(-1, 784))
            l = self.loss(output, label)
            pred = output.softmax(dim=1).argmax(dim=1)
            correct += (pred == label).sum().item()
            t += l.item()
            val_progress.set_postfix(
                loss=t / (j + 1), accuracy=correct / ((j + 1) * 1000)
            )
        t /= len(self.validloader)
        return t

In [4]:
model = Model().to("cuda")
model.train_loop("No-Regularization", 50)

Epoch 27 - Validation: 100%|██████████| 10/10 [00:00<00:00, 137.06it/s, accuracy=0.97, loss=0.156]


Learning Rate changed to 0.0005


Epoch 43 - Validation: 100%|██████████| 10/10 [00:00<00:00, 184.72it/s, accuracy=0.976, loss=0.147]


Learning Rate changed to 0.00025


Epoch 49 - Validation: 100%|██████████| 10/10 [00:00<00:00, 178.68it/s, accuracy=0.976, loss=0.149]


In [5]:
model = Model().to("cuda")
model.train_loop("L1-Regularization", 50, l1=True)

Epoch 32 - Validation: 100%|██████████| 10/10 [00:00<00:00, 168.36it/s, accuracy=0.971, loss=0.125]


Learning Rate changed to 0.0005


Epoch 49 - Validation: 100%|██████████| 10/10 [00:00<00:00, 182.77it/s, accuracy=0.979, loss=0.0885]


In [6]:
model = Model().to("cuda")
model.optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
model.train_loop("L2-Regularization", 50)

Epoch 49 - Validation: 100%|██████████| 10/10 [00:00<00:00, 54.27it/s, accuracy=0.976, loss=0.155]


In [7]:
from torchvision import transforms


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1)),
    transforms.ToTensor()
])

x_train_transformed = torch.stack([transform(img) for img in x_train.unsqueeze(1)])
x_test_transformed = torch.stack([transform(img) for img in x_test.unsqueeze(1)])

In [8]:
model = Model().to("cuda")
model.trainloader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(x_train_transformed, y_train),
    batch_size=1000,
    shuffle=True,
)
model.validloader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(x_test_transformed, y_test),
    batch_size=1000,
    shuffle=True,
)

model.train_loop("DataAug", 50)

Epoch 31 - Validation: 100%|██████████| 10/10 [00:00<00:00, 170.30it/s, accuracy=0.78, loss=0.91]


Learning Rate changed to 0.0005


Epoch 47 - Validation: 100%|██████████| 10/10 [00:00<00:00, 178.09it/s, accuracy=0.774, loss=1.23]


Learning Rate changed to 0.00025


Epoch 49 - Validation: 100%|██████████| 10/10 [00:00<00:00, 185.19it/s, accuracy=0.776, loss=1.25]


In [9]:
model = Model().to("cuda")
model.train_loop("EarlyStopping", 50, early_stopping=True)

Epoch 10 - Validation: 100%|██████████| 10/10 [00:00<00:00, 169.00it/s, accuracy=0.974, loss=0.103]

No improvement, early stopping


In [10]:
model = Model(dropout=0.5).to("cuda")
model.train_loop("Dropout", 100)

Epoch 99 - Validation: 100%|██████████| 10/10 [00:00<00:00, 116.92it/s, accuracy=0.856, loss=0.415]
